<a href="https://colab.research.google.com/github/hyphen-ani/fine-tuning-llamma/blob/main/Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 432.3/432.3 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 75.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.4/566.4 kB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.5/376.5 kB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 915.7/915.7 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.3/188.3 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from unsloth import FastLanguageModel
import torch

In [7]:
MAX_SEQUENCE_LENGTH = 2048
D_TYPE = None # Depending on the GPU, auto-configs it
LOAD_IN_4BIT = True

In [8]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = 'unsloth/Llama-3.2-3B-Instruct',
    max_seq_length = MAX_SEQUENCE_LENGTH, # Context-window length
    dtype = D_TYPE,
    load_in_4bit = LOAD_IN_4BIT
)

==((====))==  Unsloth 2026.2.1: Fast Llama patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.563 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [9]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None
)

Unsloth 2026.2.1 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [10]:
from datasets import load_dataset
dataset = load_dataset("ServiceNow-AI/R1-Distill-SFT", 'v0', split="train")

README.md: 0.00B [00:00, ?B/s]

v0/train-00000-of-00003.parquet:   0%|          | 0.00/180M [00:00<?, ?B/s]

v0/train-00001-of-00003.parquet:   0%|          | 0.00/187M [00:00<?, ?B/s]

v0/train-00002-of-00003.parquet:   0%|          | 0.00/188M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/171647 [00:00<?, ? examples/s]

In [15]:
import pandas as pd

In [ ]:
x = pd.DataFrame(dataset)
x[:10]

In [ ]:
prompt = """You are a reflective assistant engaging in thoroughly, iterative reasoning, mimicking human stream of consciousness thinking. Your approach emphasizes exploration, self-doubt, and continous refinement before coming up with an answer.
<problem>
{}
</problem>

{}
{}
"""
EOS_TOKEN = tokenizer.eos_token

def formatting_prompt(examples):
  problems = examples["problem"]
  thoughts = examples["reannotated_assistant_content"]
  solutions = examples["solution"]
  texts = []

  for problem, thought, solution in zip(problems, thoughts, solutions):
    text = prompt.format(problem, thought, solution)+EOS_TOKEN
    texts.append(text)


  return {"text": texts}

dataset = dataset.map(formatting_prompt, batched = True)

In [23]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = MAX_SEQUENCE_LENGTH,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "fine-tuned-output",
        report_to = "none",
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/171647 [00:00<?, ? examples/s]

In [ ]:
trainer_stats = trainer.train()

In [38]:
from unsloth.chat_templates import get_chat_template
from transformers import TextIteratorStreamer
import threading

sys_prompt = """You are a reflective assistant engaging in thoroughly, iterative reasoning, mimicking human stream of consciousness thinking. Your approach emphasizes exploration, self-doubt, and continous refinement before coming up with an answer. Think internally and provide only the final answer.
<problem>
{}
</problem>
"""

message = sys_prompt.format("How many r's are present in the word 'strawberry?'")
tokenizer = get_chat_template(
  tokenizer,
  chat_template = "llama-3.1",
)

FastLanguageModel.for_inference(model)

messages = [
    {"role": "user", "content": message},
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt=True,
    return_tensors="pt",
).to("cuda")

streamer = TextIteratorStreamer(
    tokenizer,
    skip_prompt=True,
    skip_special_tokens=True,
)

generation_kwargs = dict(
    input_ids=inputs,
    max_new_tokens=1024,
    use_cache=True,
    temperature=1.5,
    min_p=0.1,
    streamer=streamer,
)

thread = threading.Thread(
    target=model.generate,
    kwargs=generation_kwargs,
)
thread.start()

for token in streamer:
    print(token, end="", flush=True)

# outputs = model.generate(input_ids = inputs, max_new_tokens = 1024, use_cache = True, temperature=1.5, min_p =0.1)
# response = tokenizer.batch_decode(outputs)

First, I need to figure out where 'r' might be in the word strawberry'. Let me break it down:

- S: No 'r's here.
- T: No 'r's here.
- R: Wait, maybe strawberry' contains an 'r'. It seems that the 'r' is part of the word "strawberry".
  
Considering the structure of the word, strawberry', it contains multiple syllables:
  
- 'S-tra'
- 'war'
- 'berry'

Now, focusing on 'war' and 'berry':

- 'War' has a 'W', an 'A', an 'R', and a 'Y'. The 'R' is indeed present here.
  
- 'Berry' has a 'B', an 'E', a 'R', and a 'Y'. The 'R' is present here too.

So, in total, there are 2 'r's in the word strawberry'.
  
 I'm confident that this answer is correct because I double-checked each syllable of the word to ensure I didn't miss any 'r's.
  
Alternatively, another way to determine the number of 'r's is to look for 'rr'. 

In strawberry', I found the following words where 'rr' could appear:

1. Straw: Contains an 'r'.
2. Berry: Contains an 'rr'.

Checking each one:
  
- 'Straw': There's only one 'r'

In [ ]:
print(response[0])

In [42]:
save_dir = "./llama3-3B-KServe"

model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

('./llama3-3B-KServe/tokenizer_config.json',
 './llama3-3B-KServe/special_tokens_map.json',
 './llama3-3B-KServe/chat_template.jinja',
 './llama3-3B-KServe/tokenizer.json')

In [43]:
from huggingface_hub import login, upload_folder
login()

In [ ]:
upload_folder(folder_path="./llama3-3B-KServe", repo_id="<YOUR_REPO_ID>", repo_type="model")